In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import os
import openai

from data_apps_aws.password_manager import get_api_token
from data_apps_aws.utils import make_outside_legend

In [ ]:
plt.rcParams['figure.figsize'] = 14, 6
sns.set()

In [ ]:
openai.api_key = get_api_token('openai')

In [ ]:
# !pip install openai

In [ ]:
def gpt3(prompt, engine='davinci', response_length=64,
         temperature=0.7, top_p=1, frequency_penalty=0, presence_penalty=0,
         start_text='', restart_text='', stop_seq=[]):
    
    response = openai.Completion.create(
        prompt=prompt + start_text,
        engine=engine,
        max_tokens=response_length,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        stop=stop_seq,
    )
    
    answer = response.choices[0]['text']
    new_prompt = prompt + start_text + answer + restart_text
    
    return answer, new_prompt

In [ ]:
def get_company_info(this_company):

    example_prompt = \
    f"""Company info for Amazon: It focuses on e-commerce, cloud computing, digital streaming, and artificial intelligence.
Company info for Apple: It makes software, mobile apps, personal computers, and home appliances.
Company info for Microsoft: It makes computer software, consumer electronics, personal computers, and online services.
Company info for Tesla: It makes electric cars, solar panels, and battery products.
Company info for {this_company}:"""
    
    answer, new_prompt = gpt3(example_prompt, temperature=0, top_p=0.19, stop_seq=['Company info for'])
    
    return answer

In [ ]:
company_list = ['SAP', 'Scalable Capital', 'Kaufhof', 'Altana', 'Schering', 'Vitesco', 'Deutsche Bahn', 'HelloFresh', 'Porsche']

In [ ]:
for this_company in company_list[0:1]:
    
    answer = get_company_info(this_company)
    print(f'{this_company}: {answer}')

### FAQ Classifications

In [ ]:
faqs_raw = pd.read_csv('assets/scalable_capital_FAQs_ENG.csv')

In [ ]:
faqs_raw

In [ ]:
faqs_raw['metadata'] = faqs_raw['Category'] + ' ' + faqs_raw['Topic']
faqs_raw['text'] = faqs_raw['FAQ'] + ' ' + faqs_raw['Answer']

Write to disk

In [ ]:
df_to_json = faqs_raw.loc[:, ['text', 'metadata']]
data = df_to_json.to_json(orient='records', lines=True)
data = data[:-1] # remove \n at end of string

In [ ]:
with open('assets/scalable_capital_FAQs_ENG.jsonl', 'w') as f:
    print(data, file=f)

In [ ]:
import jsonlines

def check_jsonl_file(fname):
    
    with jsonlines.open(fname) as reader:
        for obj in reader:
            obj['metadata']

In [ ]:
def search_response_to_df(response):
    
    all_entry_list = []
    
    for this_entry in response.data:
        
        all_entry_list.append(pd.DataFrame.from_dict({0: dict(this_entry)}, orient='index'))
        
    out_df = pd.concat(all_entry_list, axis=0).loc[:, ['score', 'metadata', 'text']].sort_values('score', ascending=False).reset_index(drop=True)
        
    return out_df

In [ ]:
check_jsonl_file('assets/scalable_capital_FAQs_ENG.jsonl')

In [ ]:
file_upload_response = openai.File.create(file=open("assets/scalable_capital_FAQs_ENG_long.jsonl"), purpose="search")
file_id = file_upload_response.id

In [ ]:
this_query = 'How can I set up 2FA?'
this_query = 'Can I transfer assets to Scalable?'
this_query = 'I did not yet receive my premium?'
this_query = 'How can I set up 2FA?'
this_query = 'Do I need to use multi-factor authentication'
this_query = 'What is a ETF'
this_query = 'how much does it cost'
this_query = 'how can I see the status of my tax allowance'
this_query = 'I want to open an account for my children?'

In [ ]:
pd.set_option('display.max_colwidth', 4000)

In [ ]:
%%time
response = openai.Engine("ada").search(
    search_model="ada", 
    query=this_query, 
    max_rerank=10,
    file=file_id,
    return_metadata=True
)

out_df = search_response_to_df(response)
out_df